In [1]:
import pandas as pd
import numpy as np
import kxy

2023-11-18 00:41:41.676137: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from itertools import islice
import tscv

def generate_label(df, threshold = 0.002):
    df['label'] = 0
    df.loc[(df['target_15m'] <= -1*threshold), 'label'] = 1
    df.loc[(df['target_15m'] >= threshold), 'label'] = 2
    return df

def get_na_features(df, train_features):
    tmp = pd.DataFrame(df[train_features].isnull().sum())
    tmp = tmp[tmp[0] > 0].reset_index()
    tmp.columns = ['feat', 'cnt']
    tmp = tmp.sort_values('cnt')
    feat_groups = dict(tmp.groupby('cnt')['feat'].agg(lambda x: list(x)))
    return feat_groups

def normalize_float_columns(df, features):
  float_cols = df[features].select_dtypes(include = [float]).columns
  grouped_df = df.groupby(['token'])
  for col in float_cols:
      df[col] = grouped_df[col].transform(lambda x: (x - x.mean()) / (x.std()))
  df[float_cols] = (df[float_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0))
  return df

class Params: pass
param = Params()

In [2]:
df = pd.read_feather('../data/df_btc_eth_with_features.feather')
cols_to_drop = ['open_time', 'close_time', 'ignore',
                'create_time', 'symbol', 'returns', 'returns_5m',
                'open', 'high', 'low', 'close', 'target_15m', 'label']

In [3]:
start_time = df['open_time'].min()
end_time = df['open_time'].max()
dates = df['open_time'].unique()
n = len(dates)
train_idx = int(0.7 * n)
valid_idx = int(0.2 * n)
train_end = dates[train_idx]
valid_end = dates[valid_idx]
train_df = df.loc[df['open_time'] < train_end].reset_index(drop=True)

In [6]:
train_features = [x for x in df.columns if (x not in cols_to_drop)]

train_df['token'] = train_df['token'].astype('category').cat.codes
object_cols = train_df[train_features].select_dtypes(include=object).columns
train_df[object_cols] = train_df[object_cols].astype(float)

nan_features = get_na_features(train_df, train_features)
grouped_train = train_df.groupby(['token'])
for k, v in nan_features.items():
    for value in v:
        train_df[value] = grouped_train[value].transform(lambda x: x.ffill().fillna(0.0))

feature_cols = pd.DataFrame(train_features)
dtype_df = pd.DataFrame(train_df[train_features].select_dtypes(exclude=[float]).columns)
train_features = [x for x in train_features if x not in dtype_df.values]

In [8]:
float_cols = train_df[train_features].select_dtypes(include = [float]).columns
grouped_df = train_df.groupby(['token'])
train_df[float_cols] = (train_df[float_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0))